In [21]:
import numpy as np

In [22]:
from sklearn.tree import DecisionTreeClassifier
from interpret.glassbox import ExplainableBoostingClassifier
from interpret.glassbox import DecisionListClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB


clfs = {
    "CART": DecisionTreeClassifier(random_state=1234),
    "EBM": ExplainableBoostingClassifier(),
    "LR_l2": LogisticRegression(penalty="l2",random_state=1234),
    "GNB": GaussianNB(),
    "LR": LogisticRegression(penalty="none", random_state=1234),
    "DL": DecisionListClassifier(random_state=1234)
    
}

In [23]:
results = np.load('test_results/auc/auc_results.npy')
print("\nScores:\n", results.shape)


Scores:
 (6, 1, 10)


In [24]:
mean_results = np.mean(results, axis=2).T
print("\nMean results:\n", mean_results)


Mean results:
 [[0.75215491 0.79416286 0.76732973 0.69672895 0.76745251 0.73260328]]


In [25]:
from scipy.stats import rankdata
ranks = []
for mr in mean_results:
    ranks.append(rankdata(mr).tolist())
ranks = np.array(ranks)
print("\nRanks:\n", ranks)


Ranks:
 [[3. 6. 4. 1. 5. 2.]]


In [26]:
mean_ranks = np.mean(ranks, axis=0)
mean_ranks

array([3., 6., 4., 1., 5., 2.])

## Wilcoxon's rank statistical test

In [27]:
from scipy.stats import ranksums

alfa = .05
w_statistic = np.zeros((len(clfs), len(clfs)))
p_value = np.zeros((len(clfs), len(clfs)))

for i in range(len(clfs)):
    for j in range(len(clfs)):
        w_statistic[i, j], p_value[i, j] = ranksums(ranks.T[i], ranks.T[j])

In [28]:
from tabulate import tabulate

headers = list(clfs.keys())
names_column = np.expand_dims(np.array(list(clfs.keys())), axis=1)
w_statistic_table = np.concatenate((names_column, w_statistic), axis=1)
w_statistic_table = tabulate(w_statistic_table, headers, floatfmt=".2f")
p_value_table = np.concatenate((names_column, p_value), axis=1)
p_value_table = tabulate(p_value_table, headers, floatfmt=".2f")
print("\nw-statistic:\n", w_statistic_table, "\n\np-value:\n", p_value_table)


w-statistic:
          CART    EBM    LR_l2    GNB     LR     DL
-----  ------  -----  -------  -----  -----  -----
CART     0.00  -1.00    -1.00   1.00  -1.00   1.00
EBM      1.00   0.00     1.00   1.00   1.00   1.00
LR_l2    1.00  -1.00     0.00   1.00  -1.00   1.00
GNB     -1.00  -1.00    -1.00   0.00  -1.00  -1.00
LR       1.00  -1.00     1.00   1.00   0.00   1.00
DL      -1.00  -1.00    -1.00   1.00  -1.00   0.00 

p-value:
          CART    EBM    LR_l2    GNB    LR    DL
-----  ------  -----  -------  -----  ----  ----
CART     1.00   0.32     0.32   0.32  0.32  0.32
EBM      0.32   1.00     0.32   0.32  0.32  0.32
LR_l2    0.32   0.32     1.00   0.32  0.32  0.32
GNB      0.32   0.32     0.32   1.00  0.32  0.32
LR       0.32   0.32     0.32   0.32  1.00  0.32
DL       0.32   0.32     0.32   0.32  0.32  1.00


In [29]:
advantage = np.zeros((len(clfs), len(clfs)))
advantage[w_statistic > 0] = 1
advantage_table = tabulate(np.concatenate(
    (names_column, advantage), axis=1), headers)
print("\nAdvantage:\n", advantage_table)


Advantage:
          CART    EBM    LR_l2    GNB    LR    DL
-----  ------  -----  -------  -----  ----  ----
CART        0      0        0      1     0     1
EBM         1      0        1      1     1     1
LR_l2       1      0        0      1     0     1
GNB         0      0        0      0     0     0
LR          1      0        1      1     0     1
DL          0      0        0      1     0     0


In [30]:
significance = np.zeros((len(clfs), len(clfs)))
significance[p_value <= alfa] = 1
significance_table = tabulate(np.concatenate(
    (names_column, significance), axis=1), headers)
print("\nStatistical significance (alpha = 0.05):\n", significance_table)


Statistical significance (alpha = 0.05):
          CART    EBM    LR_l2    GNB    LR    DL
-----  ------  -----  -------  -----  ----  ----
CART        0      0        0      0     0     0
EBM         0      0        0      0     0     0
LR_l2       0      0        0      0     0     0
GNB         0      0        0      0     0     0
LR          0      0        0      0     0     0
DL          0      0        0      0     0     0
